In [1]:
import asyncio

async def count():
    print("One")
    await asyncio.sleep(1)
    print("Two")

async def main():
    await asyncio.gather(count(), count(), count())


import time
s = time.perf_counter()
await main()
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

One
One
One
Two
Two
Two
executed in 1.00 seconds.


In [2]:
# import the tidy3d package and configure it with your API key
import numpy as np
import tidy3d as td
import tidy3d.web as web
from functools import partial
from concurrent.futures import ThreadPoolExecutor
# web.configure("YOUR API KEY") # if authentication needed, uncomment this line and paste your API key here

# set up global parameters of simulation ( speed of light / wavelength in micron )
freq0 = td.C_0 / 0.75

# create structure - a box centered at 0, 0, 0 with a size of 1.5 micron and permittivity of 2
square = td.Structure(
    geometry=td.Box(center=(0, 0, 0), size=(1.5, 1.5, 1.5)),
    medium=td.Medium(permittivity=2.0)
)

# create source - A point dipole source with frequency freq0 on the left side of the domain
source = td.PointDipole(
    center=(-1.5, 0, 0),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=freq0 / 10.0),
    polarization="Ey",
)

# create monitor - Measures electromagnetic fields within the entire domain at z=0
monitor = td.FieldMonitor(
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="fields",
    colocate=True,
)

simulations = list()
for wavelength_step_i in [15, 20, 25]:
    # Initialize simulation - Combine all objects together into a single specification to run
    sim = td.Simulation(
        size=(4, 3, 3),
        grid_spec=td.GridSpec.auto(min_steps_per_wvl=wavelength_step_i),
        structures=[square],
        sources=[source],
        monitors=[monitor],
        run_time=120/freq0,
    )
    simulations.append(sim)

print(f"simulation grid is shaped {sim.grid.num_cells} for {int(np.prod(sim.grid.num_cells)/1e6)} million cells.")


i=0
kwargs_list, process_list, functions_list = list(), list(), list()
for simulation_i in simulations:
    kwargs_i = {
        "simulation": simulation_i,
        "task_name": "quickstart_" + str(i),
        "path": "data/data_" + str(i) + ".hdf5",
        "verbose": False,
    }
    kwargs_list.append(kwargs_i)
    function_i = partial(td.web.run, **kwargs_i)
    functions_list.append(function_i)
    i+=1

maximum_threads = 8
with ThreadPoolExecutor(max_workers=maximum_threads) as executor:
    for function_i in functions_list:
        future = executor.submit(function_i)

simulation grid is shaped [179, 147, 147] for 3 million cells.


In [3]:
# import the tidy3d package and configure it with your API key
import numpy as np
import tidy3d as td
import tidy3d.web as web
# web.configure("YOUR API KEY") # if authentication needed, uncomment this line and paste your API key here

# set up global parameters of simulation ( speed of light / wavelength in micron )
freq0 = td.C_0 / 0.75

# create structure - a box centered at 0, 0, 0 with a size of 1.5 micron and permittivity of 2
square = td.Structure(
    geometry=td.Box(center=(0, 0, 0), size=(1.5, 1.5, 1.5)),
    medium=td.Medium(permittivity=2.0)
)

# create source - A point dipole source with frequency freq0 on the left side of the domain
source = td.PointDipole(
    center=(-1.5, 0, 0),
    source_time=td.GaussianPulse(freq0=freq0, fwidth=freq0 / 10.0),
    polarization="Ey",
)

# create monitor - Measures electromagnetic fields within the entire domain at z=0
monitor = td.FieldMonitor(
    size=(td.inf, td.inf, 0),
    freqs=[freq0],
    name="fields",
    colocate=True,
)

Let's make a list of simulations with different variations:

In [4]:
simulations = list()
for wavelength_step_i in [15, 20, 25]:
    # Initialize simulation - Combine all objects together into a single specification to run
    sim = td.Simulation(
        size=(4, 3, 3),
        grid_spec=td.GridSpec.auto(min_steps_per_wvl=wavelength_step_i),
        structures=[square],
        sources=[source],
        monitors=[monitor],
        run_time=120/freq0,
    )
    simulations.append(sim)

print(f"simulation grid is shaped {sim.grid.num_cells} for {int(np.prod(sim.grid.num_cells)/1e6)} million cells.")

simulation grid is shaped [179, 147, 147] for 3 million cells.


## Deploying Simulations Synchronously

If we wanted to run things synchronously, we can just iterate over a `web.run` command:

We can also create a batch:

### Multithreading

In [12]:
from tidy3d.web.synchronous.api.webapi import run
from multiprocessing import Process
from multiprocessing.dummy import Pool as ThreadPool
from functools import partial
from concurrent.futures import ThreadPoolExecutor

In [11]:
i=0
kwargs_list = list()
process_list = list()
functions_list = list()
for simulation_i in simulations:
    kwargs_i = {
        "simulation": simulation_i,
        "task_name": "quickstart_" + str(i),
        "path": "data/data_" + str(i) + ".hdf5",
        "verbose": True,
    }
    kwargs_list.append(kwargs_i)
    process_i = Process(target=run, kwargs=kwargs_i)
    process_list.append(process_i)
    function_i = partial(run, **kwargs_i)
    functions_list.append(function_i)
    i+=1
len(functions_list)

3

In [15]:
with ThreadPoolExecutor(max_workers=4) as executor:
    for function_i in functions_list:
        future = executor.submit(function_i)

16:24:18 CEST Created task 'quickstart_1' with task_id                          
              'fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934' and task_type 'FDTD'.

              View task using web UI at                                         
              ]8;id=448564;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=416410;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\taskId]8;;\]8;id=448564;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\=]8;;\]8;id=279242;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\fdve]8;;\]8;id=448564;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\-62bce2ff-e8]8;;\
              ]8;id=448564;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\c3-4da9-a3ba-15a664489934']8;;\.

              Created task 'quickstart_0' with task_id                          
              'fdve-96afca91-7595-4c27-97aa-b091d927ebba' and task_type 'FDTD'.

              View task using web UI at                                         
              ]8;id=488596;https://tidy3d.simulation.cloud/workbench?taskId=fdve-96afca91-7595-4c27-97aa-b091d927ebba\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=470115;https://tidy3d.simulation.cloud/workbench?taskId=fdve-96afca91-7595-4c27-97aa-b091d927ebba\taskId]8;;\]8;id=488596;https://tidy3d.simulation.cloud/workbench?taskId=fdve-96afca91-7595-4c27-97aa-b091d927ebba\=]8;;\]8;id=97899;https://tidy3d.simulation.cloud/workbench?taskId=fdve-96afca91-7595-4c27-97aa-b091d927ebba\fdve]8;;\]8;id=488596;https://tidy3d.simulation.cloud/workbench?taskId=fdve-96afca91-7595-4c27-97aa-b091d927ebba\-96afca91-75]8;;\
              ]8;id=488596;https://tidy3d.simulation.cloud/workbench?taskId=fdve-96afca91-7595-4c27-97aa-b091d927ebba\95-4c27-97aa-b091d927ebba']8;;\.

Output()

...

Output()

16:24:38 CEST status = success

              View simulation result at                                         
              ]8;id=599292;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=870150;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\taskId]8;;\]8;id=599292;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\=]8;;\]8;id=315673;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\fdve]8;;\]8;id=599292;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\-62bce2ff-e8]8;;\
              ]8;id=599292;https://tidy3d.simulation.cloud/workbench?taskId=fdve-62bce2ff-e8c3-4da9-a3ba-15a664489934\c3-4da9-a3ba-15a664489934']8;;\.

Output()

16:24:43 CEST loading simulation from data/data_1.hdf5

In [9]:
pool = ThreadPool(4)
results = pool.map(functions_list, process_list)

NameError: name 'my_function' is not defined

In [16]:
#[run(simulation_i, task_name="quickstart", path="data/data.hdf5", verbose=True) for simulation_i in simulations]

## Deploying Simulations Asynchronously

Now, let's explore multiple ways of running this simulation asynchronously:

In [13]:
# from tidy3d.web.asynchronous.api.asynchronous import run

ModuleNotFoundError: No module named 'tidy3d.web.asynchronous.api.synchronous'

In [7]:
[run(simulation_i, task_name="quickstart", path="data/data.hdf5", verbose=True) for simulation_i in simulations]

[<coroutine object run at 0x7c99328033e0>,
 <coroutine object run at 0x7c99328034c0>,
 <coroutine object run at 0x7c99328035a0>]

In [8]:
async def main():
    await asyncio.gather(*[run(simulation_i, task_name="quickstart", path="data/data.hdf5", verbose=True) for simulation_i in simulations])
await main()

15:13:15 CEST Created task 'quickstart' with task_id                            
              'fdve-45ab2fff-e3f8-4edd-8d23-8ac772f1d802' and task_type 'FDTD'.

              View task using web UI at                                         
              ]8;id=398906;https://tidy3d.simulation.cloud/workbench?taskId=fdve-45ab2fff-e3f8-4edd-8d23-8ac772f1d802\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=383702;https://tidy3d.simulation.cloud/workbench?taskId=fdve-45ab2fff-e3f8-4edd-8d23-8ac772f1d802\taskId]8;;\]8;id=398906;https://tidy3d.simulation.cloud/workbench?taskId=fdve-45ab2fff-e3f8-4edd-8d23-8ac772f1d802\=]8;;\]8;id=462246;https://tidy3d.simulation.cloud/workbench?taskId=fdve-45ab2fff-e3f8-4edd-8d23-8ac772f1d802\fdve]8;;\]8;id=398906;https://tidy3d.simulation.cloud/workbench?taskId=fdve-45ab2fff-e3f8-4edd-8d23-8ac772f1d802\-45ab2fff-e3]8;;\
              ]8;id=398906;https://tidy3d.simulation.cloud/workbench?taskId=fdve-45ab2fff-e3f8-4edd-8d23-8ac772f1d802\f8-4edd-8d23-8ac772f1d802']8;;\.

Output()

15:13:18 CEST status = queued

              To cancel the simulation, use 'web.abort(task_id)' or             
              'web.delete(task_id)' or abort/delete the task in the web UI.     
              Terminating the Python script will not stop the job running on the
              cloud.

...

Output()

15:14:18 CEST status = success

              View simulation result at                                         
              ]8;id=429159;https://tidy3d.simulation.cloud/workbench?taskId=fdve-77064240-2682-4d8c-925b-0f9c09c8372b\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=399531;https://tidy3d.simulation.cloud/workbench?taskId=fdve-77064240-2682-4d8c-925b-0f9c09c8372b\taskId]8;;\]8;id=429159;https://tidy3d.simulation.cloud/workbench?taskId=fdve-77064240-2682-4d8c-925b-0f9c09c8372b\=]8;;\]8;id=285343;https://tidy3d.simulation.cloud/workbench?taskId=fdve-77064240-2682-4d8c-925b-0f9c09c8372b\fdve]8;;\]8;id=429159;https://tidy3d.simulation.cloud/workbench?taskId=fdve-77064240-2682-4d8c-925b-0f9c09c8372b\-77064240-26]8;;\
              ]8;id=429159;https://tidy3d.simulation.cloud/workbench?taskId=fdve-77064240-2682-4d8c-925b-0f9c09c8372b\82-4d8c-925b-0f9c09c8372b']8;;\.

Output()

15:14:20 CEST loading simulation from data/data.hdf5